In [1]:
import numpy as np
import scipy.io.wavfile
import os.path

In [2]:
def HPS(w,signal):
    signal=np.array(signal).astype(np.float64)
    if signal.ndim>1:
        signal = [s[0] for s in signal]
    
    men_min=70
    freq_middle=160
    
    parts=[]
    downsamples=5
    signal_len=len(signal)
    frames=5
    
    
    for f in range(frames):
        sig=signal[int(f*signal_len/frames) : int((f+1)*signal_len/frames)]
        freq=np.fft.fftfreq(len(sig),1/w)
        
        sig*=np.hamming(len(sig))
        fft=abs(np.fft.fft(sig))
        fft[freq<men_min]=0

        fft2=np.copy(fft)
        for i in range(downsamples):
            tmp=np.copy(fft[::(i+1)])
            fft2=fft2[:len(tmp)]
            fft2*=tmp
        
        index=np.argmax(fft[:len(fft2)]*fft2)
        parts.append(freq[index]>freq_middle)
        
    if(sum(parts)>frames/2):
        return 'K'
    else:
        return 'M'

In [3]:
def num_generator(i):
    if i < 10:
        return "00"+str(i)
    elif i<100:
        return "0"+str(i)
    else:
        return "err"

def main():
    folder_na_dane="dane"
    matrix_of_res=[[0,0],[0,0]]
    options=['K','M']
    
    good=0
    bad=0
    n=100
    
    for i in range(n+1):
        good_path=0
        for s in options:
            num=num_generator(i)
            path=folder_na_dane+"/"+num+"_"+s+".wav"
            if os.path.exists(path):
                good_path=1
                break
        if good_path==1:
            w, signal = scipy.io.wavfile.read(path)
            res=HPS(w,signal)
            
            if  (s=='K' and s==res):
                matrix_of_res[0][0]+=1
            elif(s=='K' and s!=res):
                matrix_of_res[0][1]+=1
            elif(s=='M' and s==res):
                matrix_of_res[1][1]+=1
            elif(s=='M' and s!=res):
                matrix_of_res[1][0]+=1
            
            if(s==res):
                good+=1
            else:
                bad+=1
    print(options)
    for i in range(2):
        print(options[i],matrix_of_res[i])
    print(good,bad,'\n')

In [4]:
if __name__ == '__main__':
    main()

<ipython-input-3-9c30da10307b>:27: WavFileWarning: Reached EOF prematurely; finished at 613916 bytes, expected 613924 bytes from header.
  w, signal = scipy.io.wavfile.read(path)
<ipython-input-3-9c30da10307b>:27: WavFileWarning: Chunk (non-data) not understood, skipping it.
  w, signal = scipy.io.wavfile.read(path)


['K', 'M']
K [42, 4]
M [5, 40]
82 9 

